# **Pretraining & Fine-tuning**

### **Pretraining(사전 훈련)**  
방대한 양의 데이터로 모델을 훈련하여 언어에 대한 기본적인 지식을 얻게 하는 것  

### **Fine-tuning(미세 조정)**  
사전 훈련된 모델을 (주로 새로운 task를 수행하게 하기 위해) 새로운 데이터로 이어서 훈련하는 것


# **week10: HuggingFace tutorial**

※ HuggingFace NLP Course의 3과 내용을 재구성  
원본: [Huggingface - Learn - NLP Course - Chapter 3](https://huggingface.co/learn/nlp-course/en/chapter3/1?fw=pt)

In [ ]:
# !pip3 install torch transformers evaluate datasets accelerate -U # 환경에 따라 설치 필요

In [3]:
# import warnings
# from transformers import logging

# 로그와 경고 메시지를 무시하고 싶다면 실행
# warnings.filterwarnings(action='ignore')
# logging.set_verbosity_error()

## Dataset / Preprocessing

**AutoModelFor~**  
task별 head가 추가된 모델  
불러오는 모델이 해당 head로 훈련되지 않았다면, **랜덤 초기화**된 head가 추가된다.  
→  head가 분류 능력을 얻도록 훈련하여 써야 한다.


모델 훈련 과정 한눈에 보기
- 샘플 수: 2

```python
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()
```

#### 데이터셋 불러오기

- **Dataset**
    - 구조화된 데이터, 행/열로 데이터에 접근 가능
- **DatasetDict**
    - 주로 'train', 'test' 등의 split을 담는 Dataset용 딕셔너리
- **load_dataset()**
    - HuggingFace에 업로드된 데이터셋을 불러와서 사용 가능

◎ **Dataset, DatasetDict** 관련 정보  
[HuggingFace Documentations - Datasets - References - Main Classes](https://huggingface.co/docs/datasets/v2.19.0/en/package_reference/main_classes)

◎ **load_dataset()** 관련 정보  
[HuggingFace Documentations - Datasets - How-to-Guides - Load](https://huggingface.co/docs/datasets/loading)

실습 데이터셋: GLUE - MRPC  
- GLUE(General Language Understanding Evaluation)   
  - 전반적인 자연어 이해를 평가하는 지표 모음
- MRPC(Microsoft Research Paraphrase Corpus)  
  - 두 문장이 paraphrase 관계인지 예측하는 task
  
[HuggingFace Datasets - nyu-mll/glue](https://huggingface.co/datasets/nyu-mll/glue)  
GLUE 홈페이지: [GLUE Benchmark](https://gluebenchmark.com/)

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0] # 예시 샘플

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [6]:
raw_train_dataset.features # 데이터셋 구성요소(features)

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

#### 데이터셋 전처리

In [7]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]


```python
# tokenizer는 여러 문장을 한꺼번에 처리할 수 있다.
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])
```

※ 주의: 모델이 두 문장의 관계를 예측하게 하려면, 두 문장이 pair라는 정보를 입력해야 한다.  
→ **token_type_ids**: 각 token이 둘 중 어느 문장의 일부인지 알려준다.

In [8]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
print(tokenizer.convert_ids_to_tokens(inputs["input_ids"]))

['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']


```python
['[CLS]', 'this', 'is', 'the', 'first', 'sentence', '.', '[SEP]', 'this', 'is', 'the', 'second', 'one', '.', '[SEP]']
[      0,      0,    0,     0,       0,          0,   0,       0,      1,    1,     1,        1,     1,   1,       1]
```

※ 모델에 따라 토크나이저가 token_type_ids를 출력하지 않을 수도 있다.  
∵ 문장 pair를 처리하도록 사전 훈련된 모델만이, token_type_ids를 처리할 수 있다.

문장들을 한꺼번에 pair로 토크나이저에 입력하려면...  

```python
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)
```
위 코드대로 할 수도 있지만, 대신 map() 메소드를 사용한다.  
  
  
**Dataset.map()**
- Dataset 객체의 각 샘플에 입력받은 함수를 적용한다.
- 장점: 출력 결과가 **Dataset으로 유지**된다.(위 코드는 dict를 출력한다.)  
  - **다른 처리**를 위해 map()을 또 적용할 수 있다.  
  - **메모리**에 데이터를 다 올려둘 필요 없이, 불러오려는 일부분만 올릴 수 있다.

> This works well, but it has the disadvantage of
returning a dictionary (with our keys, input_ids, attention_mask, and token_type_ids, and values that are lists of lists). It will also only work if you have enough RAM to store your whole dataset during the tokenization (whereas the datasets from the 🤗 Datasets library are Apache Arrow files stored on the disk, so you only keep the samples you ask for loaded in memory). To keep the data as a dataset, we will use the Dataset.map() method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset...

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# batch_size를 인자로 넣어 batch 크기를 조절할 수 있다.(기본값 8)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

#### Dynamic Padding

**Collate Function**  
- batch 내의 샘플들을 하나로 묶어주는 함수  
- PyTorch의 경우, Numpy array, list 등의 샘플들을 PyTorch tensor로 묶어줌


**DataCollatorWithPadding**
- Transformers 라이브러리에서, collate function의 역할을 수행
- 다른 길이의 샘플들을 묶기 위해, batch 내 최대 길이로 padding  

◎ **Data Collator** 관련 정보  
[HuggingFace Documentation - transformers - API - Data Collator](https://huggingface.co/docs/transformers/main_classes/data_collator)


In [11]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
# collate function 적용 전
samples = tokenized_datasets["train"][:8]

# 모델이 입력받는 feature만 남기고 제거
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}

# 다른 길이의 list
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

In [13]:
# collate function 적용 후
batch = data_collator(samples)

# 같은 길이의 tensor
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

## Trainer 사용

실습 모델: bert-base-uncased  
[google-bert/bert-base-uncased](https://huggingface.co/google-bert/bert-base-uncased)

#### Trainer, TrainingArguments

위에서 한 것


```python
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
```



- **Trainer**
  - 모델, 토크나이저, 데이터셋 등을 전달받아 모델 훈련 과정을 전반적으로 수행
- **TrainingArguments**
  - 체크포인트 저장, 로그 출력 등의 추가 설정 및 하이퍼파라미터 등을 묶어서 Trainer에 전달

◎ **Trainer, TrainingArguments** 관련 정보  
[HuggingFace Documentation - transformers - API - Trainers](https://huggingface.co/docs/transformers/main_classes/trainer)

In [14]:
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# TrainingArgument 설정, 체크포인트 저장 경로 이외에는 모두 기본값
training_args = TrainingArguments("test-trainer") # output_dir, 체크포인트 저장 경로

# Trainer 설정
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-b375384048a0>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


훈련을 실제로 진행하려면 아래 코드만 실행하면 된다.
```python
trainer.train()
```

##### 자주 사용되는 TrainingArguments

###### **주요 Hyperparameter**

- num_train_epochs
  - 훈련 epoch 수, 기본값 3
- per_device_train_batch_size  
  - train에 사용할 데이터셋의 batch 크기, 기본값 8
- pre_device_eval_batch_size
  - evaluation에 사용할 데이터셋의 batch 크기, 기본값 8

###### **체크포인트 저장 관련**

- save_strategy
  - 체크포인트 저장 방식, 기본값 "steps"
    - "steps": 지정한 save_steps마다 저장
    - "epoch": epoch마다 저장
    - "no": 훈련 도중에는 저장하지 않음  
      (훈련 종료 후 trainer.save_model()로 저장 가능)
- save_steps
  - (save_strategy가 "steps"일 때) 저장 주기, 기본값 500
- save_total_limit
  - 저장할 최신 체크포인트 수(나머지는 삭제), 기본값 없음(다 저장)
- load_best_model_at_end
  - True로 설정되면 evaluation 결과가 가장 좋았던 체크포인트도 추가로 남김, 기본값 False

- 예시
  - save_strategy="epoch"
  - save_total_limit=**1**
  - load_best_model_at_end=**True**
  - num_train_epochs=5
  - 최고 성능 체크포인트: 3 epoch의 체크포인트  

  → 3 epoch(**최고 성능**), 5 epoch(마지막 **1**개)의 체크포인트 저장

###### **Evalutaion 관련**

- do_eval
  - 훈련 중간 evaluation 수행 여부, 기본값 False
- eval_strategy
  - 훈련 중간 evaluation 수행 방식, 기본값 "no"
    - "steps": 지정한 eval_steps마다 evaluation
    - "epoch": epoch마다 evaluation
    - "no": evaluation 수행하지 않음
- eval_steps
  - (eval_strategy가 "steps"일 때) evaluation 주기, 기본값 없음 (미설정시 logging_steps 사용)

###### **로그 출력 관련**

- logging_strategy
  - 로그 출력 방식, 기본값 "steps"
    - "steps": 지정한 logging_steps마다 로그 출력
    - "epoch": epoch마다 로그 출력
    - "no": 로그 출력하지 않음
- logging_steps
  - (logging_strategy가 "steps"일 때) 로그 출력 주기, 기본값 500.

#### 중간 평가

TrainingArguments를 기본값으로 두면, Trainer가 아래와 같이 작동한다.
- evaluation_strategy = "no"
- logging_strategy = "steps"
- logging_steps = 500  

→ 500 step마다, training loss만 계산해 출력한다.

Trainer에 **compute_metrics**를 넣어, evaluation에서 accuracy, f1 score 등의 다른 metric을 측정할 수 있다.

◎ **Evaluate** 라이브러리 관련 정보  
[HuggingFace Documentation - Evaluate](https://huggingface.co/docs/evaluate/index)

In [15]:
# 1. 모델의 validation split에 대한 output
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [16]:
import numpy as np

# 2. 모델의 예측값
preds = np.argmax(predictions.predictions, axis=-1)

In [17]:
import evaluate

# 3. 모델의 예측값들과 정답을 비교
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

In [18]:
# 1~3의 과정을 하나로 묶은 함수
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [20]:
# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# TrainingArgument 설정, 체크포인트 저장 경로 이외, evaluation 방식 이외에는 모두 기본값
training_args = TrainingArguments(
    "test-trainer", # output_dir: 체크포인트 저장 경로
    evaluation_strategy="epoch", # evaluation 수행 방식, epoch마다 수행
    report_to="none"
    )

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics # 위에서 정의한 함수를 넣어주면, evaluation 시 같이 계산
)

trainer.train() # 훈련 결과는 매번 달라질 수 있음 (시드 고정 X)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-15a2a9ce38e9>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.514786,0.776961,0.858034
2,0.504300,0.514033,0.845588,0.892675
3,0.269700,0.780903,0.838235,0.886986


TrainOutput(global_step=1377, training_loss=0.31747924095491786, metrics={'train_runtime': 303.4116, 'train_samples_per_second': 36.268, 'train_steps_per_second': 4.538, 'total_flos': 405114969714960.0, 'train_loss': 0.31747924095491786, 'epoch': 3.0})

## Trainer 구현

#### DataLoader 생성

위에서 한 것


```python
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
```



In [21]:
# 훈련에 사용되지 않는 feature 제거
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])

# 모델은 "label" 대신 "labels"를 입력받음
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# list들을 PyTorch tensor들로 변환
tokenized_datasets.set_format("torch")

# 남은 feature 확인
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True, # 순서 섞기
    batch_size=8, # ≒ Trainer의 per_device_train_batch_size
    collate_fn=data_collator # DataCollaterWithPadding
)

eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8, # ≒ Trainer의 per_device_eval_batch_size
    collate_fn=data_collator # DataCollaterWithPadding
)

# padding은 collate_fn으로 들어간 DataCollatorWithPadding이 해준다.

In [23]:
# train_dataloader의 첫 batch 확인
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 81]),
 'token_type_ids': torch.Size([8, 81]),
 'attention_mask': torch.Size([8, 81])}

#### 훈련 준비

1. 모델 불러오기
2. optimizer 생성
3. epoch 수 설정
4. learning rate scheduler 생성

In [24]:
from transformers import AutoModelForSequenceClassification

# 1. 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 첫 batch를 모델에 입력
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor(0.9363, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [25]:
from transformers import AdamW

# 2. optimizer 생성
optimizer = AdamW( # Trainer의 기본 optimizer = AdamW
    model.parameters(),
    lr=5e-5 # Trainer의 AdamW learning rate 기본값 = 0.00005
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
from transformers import get_scheduler

# 3. epoch 수 설정
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader) # train_dataloader의 길이 = batch 수

# 4. learning rate scheduler 생성
lr_scheduler = get_scheduler(
    "linear", # Trainer의 AdamW learning rate scheduler 기본값 = "linear", 0까지 선형 감소
    optimizer=optimizer,
    num_warmup_steps=0, # warmup 없이 설정된 값에서 바로 시작, 선형 감소
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


#### Training Loop

In [27]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [28]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train() # 훈련 모드
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch) # forward
        loss = outputs.loss # loss 계산
        loss.backward() # loss로부터 gradient 계산

        optimizer.step() # parameter update
        lr_scheduler.step() # scheduler에 따라 learning rate 변경
        optimizer.zero_grad() # optimizer에 누적된 gradient 초기화
        progress_bar.update(1) # 진행 상황 업데이트

  0%|          | 0/1377 [00:00<?, ?it/s]

훈련 도중에는 모델이 얼마나 잘 훈련되고 있는지는 알 수 없지만, 훈련이 끝난 후 evaluation으로 확인할 수 있다.

In [29]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval() # 평가 모드
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad(): # gradient 추적 제거 (parameter를 수정하지 않으므로)
        outputs = model(**batch) # forward

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1) # 모델의 예측값
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8725490196078431, 'f1': 0.910958904109589}

#### Accelerate 사용하기

Accelerate를 사용하여, 여러 대의 GPU로 훈련 과정을 나눠서 빠르게 수행할 수 있다.  

◎ **Accelerate** 라이브러리 관련 정보  
[HuggingFace Documentation - Accelerate](https://huggingface.co/docs/accelerate/index)


위에서 해본 Trainer 따라하기
```python
from tqdm.auto import tqdm

from transformers import AdamW
from transformers import get_scheduler
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
```



Accelerate를 사용하려면 아래처럼 수정
```python
from tqdm.auto import tqdm

from accelerate import Accelerator # 추가
from transformers import AdamW
from transformers import get_scheduler
from transformers import AutoModelForSequenceClassification

accelerator = Accelerator() # 추가

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# Accelerate가 device 배정을 알아서 해준다.

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
) # 추가: 여러 작업을 Accelerate가 알아서 수행한다.

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # loss.backward()
        accelerator.backward(loss) # 추가: loss.backward()를 대신한다.

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
```



위의 코드를 train.py라는 스크립트 파일로 만들어 실행할 수도 있다.  
1. 아래 명령어를 실행 하고, config들을 입력한다.
    ```
    accelerate config
    ```
2. 아래 명령어를 이어서 실행한다.
    ```
    accelerate launch train.py
    ```  

Jupyter Notebook에서 위의 코드를 실행하려면, training_function()로 정의한 다음 아래 코드를 실행하면 된다.
  ```python
  from accelerate import notebook_launcher

  notebook_launcher(training_function)
  ```




## 총정리

#### 데이터 준비하기

```python
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

# 데이터셋 불러오기
raw_datasets = load_dataset("glue", "mrpc")

# 토크나이저 불러오기
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 데이터셋에 토크나이저 적용
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

# Data Collator 생성
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
```


#### Trainer 사용
```python
import evaluate
from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

# metric 설정하기
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# TrainingArgument 설정하기
training_args = TrainingArguments(
    "test-trainer",
    evaluation_strategy="epoch"
    )

# Trainer 설정하기
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 훈련 진행하기
trainer.train()
```



#### Trainer 구현
```python
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader
import evaluate
from transformers import AdamW
from transformers import get_scheduler
from transformers import AutoModelForSequenceClassification

# 데이터 처리
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

# DataLoader 생성
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=8,
    collate_fn=data_collator
)

# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

# device 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 훈련 준비
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# 훈련 진행
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# 모델 평가
metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()
```
Accelerate를 사용하려면 여기에 위에서처럼 행들을 추가/제거하면 된다.